<a href="https://colab.research.google.com/github/wang201156/ycps0218/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask
!pip install pyngrok
!pip install flask_ngrok
!pip install line-bot-sdk

In [ ]:
#===================================
#for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK'))
  ngrok.connect(5000)
  run_with_ngrok(app)
#===================================

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
# Fix: Complete the import statement
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    StickerMessage,
    ImageMessage,
    VideoMessage,
    LocationMessage,
    TextMessage,
    TemplateMessage,
    ConfirmTemplate,
    MessageAction
)
from linebot.v3.messaging import (
    TemplateMessage,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

@app.route("/", methods=['GET'])
def index():
  return 'Hello'

configuration = Configuration(access_token=userdata.get("LINE_CHANNEL_access_token"))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_Channel_secret'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action=event.message.text
#=======================================================================
        if action == "confirm":
            template=ConfirmTemplate(
              text="這句話是錯的",
              actions=[
               MessageAction(label="是", text="True"),
               MessageAction(label="否", text="False")
                ]
              )
            # Fix: Correct indentation for this line
            reply = TemplateMessage(
              alt_text="確認",
              template=template
              )

#=======================================================================
        elif action == "carousel":
          # 定義 CarouselTemplate
          carousel_template = CarouselTemplate(
              columns=[
                  CarouselColumn(
                      thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                      title='夏日大作戰',
                      text='導演:細田守',
                      actions=[
                          URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0'),
                          MessageAction(label='投票', text='投夏日大作戰一票')
                      ]
                  ),
                  CarouselColumn(
                      thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/78/Promare_Key_Visual.jpg',
                      title='Promare',
                      text='導演:今石洋之',
                      actions=[
                          URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E6%99%AE%E7%BE%85%E7%B1%B3%E4%BA%9E'),
                          MessageAction(label='投票', text='投Promare一票')
                      ]
                  )
              ]
          )
          # 包裝成 TemplateMessage
          template_message = TemplateMessage(
              alt_text='這是一個輪播模板',
              template=carousel_template
          )
          reply = template_message # Add this line
#=======================================================================

        else:
            reply = TextMessage(text="Thanks you")

        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )
#=======================================================================


ngrok_start() #for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://20ea-34-106-104-167.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 07:57:47] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 07:57:48] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 07:57:56] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 07:58:11] "POST /callback HTTP/1.1" 200 -
